# Notebook error analysis 

## Manual labels of the sampled errors for each cluster

in terms of bug types and root causes

In [20]:
# import importlib
# importlib.reload(config)
# import config

path_default=WindowsPath('C:/Users/yirwa29/Downloads/Dataset-Nb')


# 1. Prepare statistics

In [39]:
import pandas as pd
import config

df_mlerr_labels = pd.read_csv(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled2_Yiran.csv'), 
                              header=0, 
                              encoding='ISO-8859-1')
df_mlerr_label_config = pd.read_csv(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled2_Config.csv'),
                                    header=0,
                                    encoding='ISO-8859-1')
df_mlerr_labels_g = df_mlerr_labels[df_mlerr_labels["nb_source"]==config.NB_SOURCE["github"]].reset_index(drop=True)
df_mlerr_labels_k = df_mlerr_labels[df_mlerr_labels["nb_source"]==config.NB_SOURCE["kaggle"]].reset_index(drop=True)
del df_mlerr_labels

In [40]:
assert(len(df_mlerr_labels_g)+len(df_mlerr_labels_k)==len(df_mlerr_labels))

In [75]:
# selected_cluster_sizes = pd.read_excel(config.path_default.joinpath("data_dedup_cluster/df_err_processed_pregroup_cluster_sizes.xlsx"))
selected_cluster_sizes_k = pd.read_excel(config.path_default.joinpath("data_dedup_cluster/df_err_processed_pregroup_cluster_sizes_k.xlsx"))
selected_cluster_sizes_g = pd.read_excel(config.path_default.joinpath("data_dedup_cluster/df_err_processed_pregroup_cluster_sizes_g.xlsx"))

In [ ]:
# need to take care of n/a!!!!!!



In [76]:
import pickle

# summarize the statistics
def statistics_labels(df_mlerr_label_config, selected_cluster_sizes, df_mlerr_labels):
    cluster_res = {}
    for label_key in df_mlerr_label_config:
        cluster_res[label_key] = {}

    for index, row in selected_cluster_sizes.iterrows():
        cluster_id = row.pregroup_cluster
        cluster_size = row.cluster_size
        cluster_errors = df_mlerr_labels[df_mlerr_labels.pregroup_cluster == cluster_id]

        for label_key in df_mlerr_label_config:
            for label_key_option in df_mlerr_label_config[label_key].dropna():
                cluster_label_key_option_size = sum(cluster_errors[label_key]==label_key_option)
                if label_key_option not in cluster_res[label_key]:
                    cluster_res[label_key][label_key_option] = {}
                cluster_res[label_key][label_key_option][cluster_id] = (cluster_label_key_option_size, len(cluster_errors), cluster_size)
    return cluster_res

def save_statistics(save_path, cluster_res):
    with open(save_path, 'wb') as handle:
        pickle.dump(cluster_res, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [80]:
cluster_res_g = statistics_labels(df_mlerr_label_config, selected_cluster_sizes_g, df_mlerr_labels_g)
save_statistics(config.path_default.joinpath("data_dedup_cluster/sample2_statistics_g.pickle"), cluster_res_g)

cluster_res_k = statistics_labels(df_mlerr_label_config, selected_cluster_sizes_k, df_mlerr_labels_k)
save_statistics(config.path_default.joinpath("data_dedup_cluster/sample2_statistics_k.pickle"), cluster_res_k)   

# 2. Analysis

In [ ]:
import pandas as pd
import config

def load_statistics(load_path):
    with open(load_path, 'rb') as handle:
        cluster_res_load = pickle.load(handle)
    return cluster_res_load

df_mlerr_label_config = pd.read_csv(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled2_Config.csv'),
                                    header=0,
                                    encoding='ISO-8859-1')

### 2.1. GitHub errors

In [86]:
cluster_res_g = load_statistics(config.path_default.joinpath("data_dedup_cluster/sample2_statistics_g.pickle"))
for label_key in df_mlerr_label_config:
    cluster_res_g[label_key] = pd.DataFrame.from_dict(cluster_res_g[label_key]).rename_axis('cluster_id').reset_index()
cluster_res_g["label_root_cause"].head()

,cluster_id,misunderstanding of data structure,misunderstanding of APIs,invalid argument,nb specific - execution order,nb specific - previous cell error,nb specific - need execute future cells,unknown,did not import,undefined variable,...,module not installed,change of environment,file/path not found,wrong implementation,misunderstanding of libraries,API change,library versions incompatible,misunderstanding of types of objects,uninitializated,error inside library
0,1431,"(0, 15, 24028)","(0, 15, 24028)","(0, 15, 24028)","(4, 15, 24028)","(5, 15, 24028)","(1, 15, 24028)","(1, 15, 24028)","(2, 15, 24028)","(2, 15, 24028)",...,"(0, 15, 24028)","(0, 15, 24028)","(0, 15, 24028)","(0, 15, 24028)","(0, 15, 24028)","(0, 15, 24028)","(0, 15, 24028)","(0, 15, 24028)","(0, 15, 24028)","(0, 15, 24028)"
1,273,"(2, 15, 5882)","(10, 15, 5882)","(0, 15, 5882)","(1, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)",...,"(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)","(0, 15, 5882)"
2,20,"(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)",...,"(14, 15, 5012)","(1, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)","(0, 15, 5012)"
3,9,"(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)",...,"(0, 15, 3238)","(0, 15, 3238)","(14, 15, 3238)","(1, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)","(0, 15, 3238)"
4,7430,"(0, 15, 1444)","(0, 15, 1444)","(0, 15, 1444)","(0, 15, 1444)","(0, 15, 1444)","(0, 15, 1444)","(3, 15, 1444)","(0, 15, 1444)","(0, 15, 1444)",...,"(1, 15, 1444)","(0, 15, 1444)","(0, 15, 1444)","(0, 15, 1444)","(4, 15, 1444)","(1, 15, 1444)","(3, 15, 1444)","(0, 15, 1444)","(0, 15, 1444)","(0, 15, 1444)"


### 2.2. Kaggle errors

In [88]:
cluster_res_k = load_statistics(config.path_default.joinpath("data_dedup_cluster/sample2_statistics_k.pickle"))
for label_key in df_mlerr_label_config:
    cluster_res_k[label_key] = pd.DataFrame.from_dict(cluster_res_k[label_key]).rename_axis('cluster_id').reset_index()
cluster_res_k["label_root_cause"].head()

,cluster_id,misunderstanding of data structure,misunderstanding of APIs,invalid argument,nb specific - execution order,nb specific - previous cell error,nb specific - need execute future cells,unknown,did not import,undefined variable,...,module not installed,change of environment,file/path not found,wrong implementation,misunderstanding of libraries,API change,library versions incompatible,misunderstanding of types of objects,uninitializated,error inside library
0,1431,"(0, 15, 1042)","(0, 15, 1042)","(0, 15, 1042)","(6, 15, 1042)","(2, 15, 1042)","(0, 15, 1042)","(1, 15, 1042)","(0, 15, 1042)","(6, 15, 1042)",...,"(0, 15, 1042)","(0, 15, 1042)","(0, 15, 1042)","(0, 15, 1042)","(0, 15, 1042)","(0, 15, 1042)","(0, 15, 1042)","(0, 15, 1042)","(0, 15, 1042)","(0, 15, 1042)"
1,273,"(0, 15, 225)","(9, 15, 225)","(2, 15, 225)","(0, 15, 225)","(1, 15, 225)","(0, 15, 225)","(1, 15, 225)","(0, 15, 225)","(0, 15, 225)",...,"(0, 15, 225)","(0, 15, 225)","(0, 15, 225)","(0, 15, 225)","(0, 15, 225)","(0, 15, 225)","(0, 15, 225)","(0, 15, 225)","(0, 15, 225)","(0, 15, 225)"
2,9,"(0, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)",...,"(0, 15, 172)","(0, 15, 172)","(15, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)","(0, 15, 172)"
3,20,"(0, 15, 124)","(0, 15, 124)","(0, 15, 124)","(0, 15, 124)","(1, 15, 124)","(0, 15, 124)","(1, 15, 124)","(0, 15, 124)","(0, 15, 124)",...,"(11, 15, 124)","(2, 15, 124)","(0, 15, 124)","(0, 15, 124)","(0, 15, 124)","(0, 15, 124)","(0, 15, 124)","(0, 15, 124)","(0, 15, 124)","(0, 15, 124)"
4,1835,"(8, 15, 64)","(6, 15, 64)","(0, 15, 64)","(0, 15, 64)","(0, 15, 64)","(0, 15, 64)","(1, 15, 64)","(0, 15, 64)","(0, 15, 64)",...,"(0, 15, 64)","(0, 15, 64)","(0, 15, 64)","(0, 15, 64)","(0, 15, 64)","(0, 15, 64)","(0, 15, 64)","(0, 15, 64)","(0, 15, 64)","(0, 15, 64)"
